In [1]:
from statsbombpy import sb

competitions = sb.competitions(fmt='json')

records = []
for key, val in competitions.items():
        # key: tuple (country, competition, season, gender)
        # val: dict with all fields
        record = {
            'competition_id': val.get('competition_id'),
            'season_id': val.get('season_id'),
            'competition_name': val.get('competition_name'),
            'competition_gender': val.get('competition_gender'),
            'country_name': val.get('country_name'),
            'competition_youth': val.get('competition_youth'),
            'competition_international': val.get('competition_international'),
            'season_name': val.get('season_name'),
            'match_updated': val.get('match_updated'),
            'match_updated_360': val.get('match_updated_360'),
            'match_available_360': val.get('match_available_360'),
            'match_available': val.get('match_available'),
            # Also include tuple key fields for reference
            'key_country': key[0] if len(key) > 0 else None,
            'key_competition': key[1] if len(key) > 1 else None,
            'key_season': key[2] if len(key) > 2 else None,
            'key_gender': key[3] if len(key) > 3 else None,
        }
        records.append(record)

records

/home/codespace/.local/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


[{'competition_id': 9,
  'season_id': 281,
  'competition_name': '1. Bundesliga',
  'competition_gender': 'male',
  'country_name': 'Germany',
  'competition_youth': False,
  'competition_international': False,
  'season_name': '2023/2024',
  'match_updated': '2024-07-15T14:15:54.671676',
  'match_updated_360': '2024-07-15T14:17:00.877356',
  'match_available_360': '2024-07-15T14:17:00.877356',
  'match_available': '2024-07-15T14:15:54.671676',
  'key_country': 'Germany',
  'key_competition': '1. Bundesliga',
  'key_season': '2023/2024',
  'key_gender': 'male'},
 {'competition_id': 9,
  'season_id': 27,
  'competition_name': '1. Bundesliga',
  'competition_gender': 'male',
  'country_name': 'Germany',
  'competition_youth': False,
  'competition_international': False,
  'season_name': '2015/2016',
  'match_updated': '2024-05-19T11:11:14.192381',
  'match_updated_360': None,
  'match_available_360': None,
  'match_available': '2024-05-19T11:11:14.192381',
  'key_country': 'Germany',
  '

In [2]:
matches = sb.matches(competition_id=9, season_id=281, fmt='json')

matches_records = []
for k, m in matches.items():
            record = {
                'match_id': m.get('match_id'),
                'match_date': m.get('match_date'),
                'kick_off': m.get('kick_off'),
                'competition_id': m.get('competition', {}).get('competition_id') if isinstance(m.get('competition'), dict) else m.get('competition_id'),
                'competition_name': m.get('competition', {}).get('competition_name') if isinstance(m.get('competition'), dict) else m.get('competition_name'),
                'country_name': m.get('competition', {}).get('country_name') if isinstance(m.get('competition'), dict) else m.get('country_name'),
                'season_id': m.get('season', {}).get('season_id') if isinstance(m.get('season'), dict) else m.get('season_id'),
                'season_name': m.get('season', {}).get('season_name') if isinstance(m.get('season'), dict) else m.get('season_name'),
                'home_team_id': m.get('home_team', {}).get('home_team_id') if isinstance(m.get('home_team'), dict) else m.get('home_team_id'),
                'home_team_name': m.get('home_team', {}).get('home_team_name') if isinstance(m.get('home_team'), dict) else m.get('home_team'),
                'home_team_gender': m.get('home_team', {}).get('home_team_gender') if isinstance(m.get('home_team'), dict) else None,
                'home_team_group': m.get('home_team', {}).get('home_team_group') if isinstance(m.get('home_team'), dict) else None,
                'home_team_country_id': m.get('home_team', {}).get('country', {}).get('id') if isinstance(m.get('home_team', {}).get('country'), dict) else None,
                'home_team_country_name': m.get('home_team', {}).get('country', {}).get('name') if isinstance(m.get('home_team', {}).get('country'), dict) else None,
                'home_manager_id': m.get('home_team', {}).get('managers', [{}])[0].get('id') if m.get('home_team', {}).get('managers') else None,
                'home_manager_name': m.get('home_team', {}).get('managers', [{}])[0].get('name') if m.get('home_team', {}).get('managers') else None,
                'away_team_id': m.get('away_team', {}).get('away_team_id') if isinstance(m.get('away_team'), dict) else m.get('away_team_id'),
                'away_team_name': m.get('away_team', {}).get('away_team_name') if isinstance(m.get('away_team'), dict) else m.get('away_team'),
                'away_team_gender': m.get('away_team', {}).get('away_team_gender') if isinstance(m.get('away_team'), dict) else None,
                'away_team_group': m.get('away_team', {}).get('away_team_group') if isinstance(m.get('away_team'), dict) else None,
                'away_team_country_id': m.get('away_team', {}).get('country', {}).get('id') if isinstance(m.get('away_team', {}).get('country'), dict) else None,
                'away_team_country_name': m.get('away_team', {}).get('country', {}).get('name') if isinstance(m.get('away_team', {}).get('country'), dict) else None,
                'away_manager_id': m.get('away_team', {}).get('managers', [{}])[0].get('id') if m.get('away_team', {}).get('managers') else None,
                'away_manager_name': m.get('away_team', {}).get('managers', [{}])[0].get('name') if m.get('away_team', {}).get('managers') else None,
                'home_score': m.get('home_score'),
                'away_score': m.get('away_score'),
                'match_status': m.get('match_status'),
                'match_status_360': m.get('match_status_360'),
                'last_updated': m.get('last_updated'),
                'last_updated_360': m.get('last_updated_360'),
                'data_version': m.get('metadata', {}).get('data_version') if isinstance(m.get('metadata'), dict) else None,
                'shot_fidelity_version': m.get('metadata', {}).get('shot_fidelity_version') if isinstance(m.get('metadata'), dict) else None,
                'xy_fidelity_version': m.get('metadata', {}).get('xy_fidelity_version') if isinstance(m.get('metadata'), dict) else None,
                'match_week': m.get('match_week'),
                'competition_stage_id': m.get('competition_stage', {}).get('id') if isinstance(m.get('competition_stage'), dict) else None,
                'competition_stage_name': m.get('competition_stage', {}).get('name') if isinstance(m.get('competition_stage'), dict) else None,
                'stadium_id': m.get('stadium', {}).get('id') if isinstance(m.get('stadium'), dict) else None,
                'stadium_name': m.get('stadium', {}).get('name') if isinstance(m.get('stadium'), dict) else None,
                'stadium_country_id': m.get('stadium', {}).get('country', {}).get('id') if isinstance(m.get('stadium', {}).get('country'), dict) else None,
                'stadium_country_name': m.get('stadium', {}).get('country', {}).get('name') if isinstance(m.get('stadium', {}).get('country'), dict) else None,
                'referee_id': m.get('referee', {}).get('id') if isinstance(m.get('referee'), dict) else None,
                'referee_name': m.get('referee', {}).get('name') if isinstance(m.get('referee'), dict) else None,
                'referee_country_id': m.get('referee', {}).get('country', {}).get('id') if isinstance(m.get('referee', {}).get('country'), dict) else None,
                'referee_country_name': m.get('referee', {}).get('country', {}).get('name') if isinstance(m.get('referee', {}).get('country'), dict) else None,
                # Add more fields as needed
            }
            matches_records.append(record)
matches_records[0]

{'match_id': 3895302,
 'match_date': '2024-04-14',
 'kick_off': '17:30:00.000',
 'competition_id': 9,
 'competition_name': '1. Bundesliga',
 'country_name': 'Germany',
 'season_id': 281,
 'season_name': '2023/2024',
 'home_team_id': 904,
 'home_team_name': 'Bayer Leverkusen',
 'home_team_gender': 'male',
 'home_team_group': None,
 'home_team_country_id': 85,
 'home_team_country_name': 'Germany',
 'home_manager_id': 1000310,
 'home_manager_name': 'Xabier Alonso Olano',
 'away_team_id': 176,
 'away_team_name': 'Werder Bremen',
 'away_team_gender': 'male',
 'away_team_group': None,
 'away_team_country_id': 85,
 'away_team_country_name': 'Germany',
 'away_manager_id': 4057,
 'away_manager_name': 'Ole Werner',
 'home_score': 5,
 'away_score': 0,
 'match_status': 'available',
 'match_status_360': 'available',
 'last_updated': '2024-05-10T16:57:53.017895',
 'last_updated_360': '2024-05-10T17:03:59.613154',
 'data_version': '1.1.0',
 'shot_fidelity_version': '2',
 'xy_fidelity_version': '2',
 

In [ ]:
events = sb.events(match_id=3895302, fmt='json')

events_records = []
for key, e in events.items():
    record = {
        'id': e.get('id'),
        'index': e.get('index'),
        'period': e.get('period'),
        'timestamp': e.get('timestamp'),
        'minute': e.get('minute'),
        'second': e.get('second'),
        'type_id': e.get('type', {}).get('id') if isinstance(e.get('type'), dict) else None,
        'type_name': e.get('type', {}).get('name') if isinstance(e.get('type'), dict) else e.get('type'),
        'possession': e.get('possession'),
        'possession_team_id': e.get('possession_team', {}).get('id') if isinstance(e.get('possession_team'), dict) else None,
        'possession_team_name': e.get('possession_team', {}).get('name') if isinstance(e.get('possession_team'), dict) else e.get('possession_team'),
        'play_pattern_id': e.get('play_pattern', {}).get('id') if isinstance(e.get('play_pattern'), dict) else None,
        'play_pattern_name': e.get('play_pattern', {}).get('name') if isinstance(e.get('play_pattern'), dict) else e.get('play_pattern'),
        'team_id': e.get('team', {}).get('id') if isinstance(e.get('team'), dict) else None,
        'team_name': e.get('team', {}).get('name') if isinstance(e.get('team'), dict) else e.get('team'),
        'player_id': e.get('player', {}).get('id') if isinstance(e.get('player'), dict) else None,
        'player_name': e.get('player', {}).get('name') if isinstance(e.get('player'), dict) else e.get('player'),
        'location': e.get('location'),
        'duration': e.get('duration'),
        'match_id': e.get('match_id'),
        # Tactics fields
        'tactics_formation': e.get('tactics', {}).get('formation') if isinstance(e.get('tactics'), dict) else None,
        'tactics_lineup': e.get('tactics', {}).get('lineup') if isinstance(e.get('tactics'), dict) else None,
        # Add more fields as needed, including nested ones
    }
    # Optionally flatten more nested fields (e.g., pass, shot, etc.)
    if 'pass' in e:
        record['pass_end_location'] = e['pass'].get('end_location')
        record['pass_outcome'] = e['pass'].get('outcome', {}).get('name') if isinstance(e['pass'].get('outcome'), dict) else e['pass'].get('outcome')
    if 'shot' in e:
        record['shot_end_location'] = e['shot'].get('end_location')
        record['shot_outcome'] = e['shot'].get('outcome', {}).get('name') if isinstance(e['shot'].get('outcome'), dict) else e['shot'].get('outcome')
        record['shot_statsbomb_xg'] = e['shot'].get('statsbomb_xg')
    # Optionally flatten lineup (store as JSON string for DataFrame compatibility)
    if 'tactics' in e and isinstance(e['tactics'], dict) and 'lineup' in e['tactics']:
        import json as _json
        record['tactics_lineup_json'] = _json.dumps(e['tactics']['lineup'])
    events_records.append(record)
events_records[0]

{'id': '794ec549-5288-4d1a-93e8-0fc6d3968784',
 'index': 1,
 'period': 1,
 'timestamp': '00:00:00.000',
 'minute': 0,
 'second': 0,
 'type_id': 35,
 'type_name': 'Starting XI',
 'possession': 1,
 'possession_team_id': 904,
 'possession_team_name': 'Bayer Leverkusen',
 'play_pattern_id': 1,
 'play_pattern_name': 'Regular Play',
 'team_id': 904,
 'team_name': 'Bayer Leverkusen',
 'player_id': None,
 'player_name': None,
 'location': None,
 'duration': 0.0,
 'match_id': 3895302,
 'tactics_formation': 3421,
 'tactics_lineup': [{'player': {'id': 8667, 'name': 'Lukáš Hrádecký'},
   'position': {'id': 1, 'name': 'Goalkeeper'},
   'jersey_number': 1},
  {'player': {'id': 27133, 'name': 'Odilon Kossonou'},
   'position': {'id': 3, 'name': 'Right Center Back'},
   'jersey_number': 6},
  {'player': {'id': 8221, 'name': 'Jonathan Tah'},
   'position': {'id': 4, 'name': 'Center Back'},
   'jersey_number': 4},
  {'player': {'id': 30606, 'name': 'Edmond Fayçal Tapsoba'},
   'position': {'id': 5, 'nam